In [ ]:
#@markdown ⬅ 點擊執行按鈕開始清理個人分類資料夾
#@markdown ---
#@markdown <br><br>
#@markdown # 使用者設定
#@markdown ---
#@title 清空分類後的個人資料夾 { display-mode: "form" }
總資料夾 = "" #@param {type:"string", placeholder:"輸入總資料夾網址"}
#@markdown - 總資料夾內應該要有名稱為你 Email 帳號的資料夾，如果沒有則執行也無效。
#@markdown - 請輸入瀏覽器網址列顯示的網址。
#@markdown - 你必須是該資料夾的編輯者或擁有者。
#@markdown - 你應該要是該資料夾下檔案的擁有者，不過腳本仍然會檢查並跳過不屬於你的檔案。
#@markdown ---
請求間隔 = 0.2 # @param {type:"slider", min:0.1, max:5, step:0.1}
#@markdown - API 請求之間的延遲（秒），以避免觸發速率限制。
#@markdown ---
自動清除垃圾桶 = False # @param {type:"boolean"}
#@markdown - 是否自動清除垃圾桶？
#@markdown ---
#@markdown <br><br>
操作確認 = "" #@param {type:"string", placeholder:"輸入您的 email 以確認操作"}
#@markdown <br><br>
# -*- coding: utf-8 -*-

# 變數名稱轉換
TARGET_BASE_FOLDER_ID = 總資料夾
DELAY_BETWEEN_REQUESTS = 請求間隔
AUTO_CLEAN = 自動清除垃圾桶
EMAIL_VERIFY = 操作確認

"""
將個人分類資料夾內的所有項目 (檔案與子資料夾) 移至垃圾桶 (Google Colab Python 版本)

此腳本會：
1. 取得目前執行者的 Email。
2. 在指定的 '總資料夾' (例如 '待刪除檔案') 中，找到以該 Email 命名的子資料夾。
3. 將該 Email 子資料夾 **內屬於你的所有項目 (檔案與子資料夾)** 移至垃圾桶。
4. **不會** 自動清空垃圾桶，您可以選擇執行下一個儲存格來清空。

**警告：此操作會將指定資料夾內的所有項目移至垃圾桶。後續可選擇是否永久清空。請謹慎使用！**
"""

# --- 安裝與載入套件 ---
# !pip install --quiet --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib tqdm

from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.auth import default
from tqdm.notebook import tqdm # 使用 notebook 版本的 tqdm
import time
import sys
import re

# --- 全域變數 ---
user_email = ''
drive_service = None
target_base_folder_name = '' # 用於顯示

# --- 輔助函數 ---

def authenticate_and_build_service():
    """處理 Colab 驗證並建立 Drive API 服務"""
    global drive_service, user_email
    try:
        print("⏳ 正在請求 Google Drive 授權...")
        auth.authenticate_user()
        creds, _ = default()
        drive_service = build('drive', 'v3', credentials=creds, cache_discovery=False)
        print("✅ 授權成功，正在建立 Drive 服務...")
        # 取得使用者資訊
        about = drive_service.about().get(fields="user").execute()
        user_email = about['user']['emailAddress']
        print(f"✅ Drive 服務建立成功，目前使用者: {user_email}")
        return True
    except Exception as e:
        print(f"❌ 驗證或建立服務失敗: {e}")
        return False

def get_folder_name(folder_id):
    """取得指定 ID 資料夾的名稱"""
    global drive_service, DELAY_BETWEEN_REQUESTS
    try:
        time.sleep(DELAY_BETWEEN_REQUESTS)
        folder = drive_service.files().get(fileId=folder_id, fields='name, mimeType, trashed').execute()
        if folder.get('mimeType') != 'application/vnd.google-apps.folder':
            print(f"  ⚠️ 提供的 ID '{folder_id}' 不是一個資料夾。")
            return None
        if folder.get('trashed', False):
            print(f"  ⚠️ 總資料夾 '{folder.get('name')}' (ID: {folder_id}) 已在垃圾桶中。")
            return None
        return folder.get('name')
    except HttpError as error:
        print(f"  ❌ 獲取資料夾名稱 (ID: {folder_id}) 時發生 API 錯誤: {error}")
        return None
    except Exception as e:
        print(f"  ❌ 獲取資料夾名稱 (ID: {folder_id}) 時發生非預期錯誤: {e}")
        return None


def find_folder_id_by_name(parent_folder_id, folder_name):
    """在指定父資料夾中尋找特定名稱的子資料夾 ID"""
    global drive_service, DELAY_BETWEEN_REQUESTS
    query = (f"'{parent_folder_id}' in parents and "
             f"name = '{folder_name}' and "
             f"mimeType = 'application/vnd.google-apps.folder' and "
             f"trashed = false")
    try:
        time.sleep(DELAY_BETWEEN_REQUESTS)
        response = drive_service.files().list(q=query,
                                              spaces='drive',
                                              fields='files(id, name)',
                                              pageSize=1).execute()
        folders = response.get('files', [])
        if folders:
            return folders[0]['id']
        else:
            return None
    except HttpError as error:
        print(f"  ❌ 在資料夾 (ID: {parent_folder_id}) 中尋找子資料夾 '{folder_name}' 時發生 API 錯誤: {error}")
        return None
    except Exception as e:
         print(f"  ❌ 在資料夾 (ID: {parent_folder_id}) 中尋找子資料夾 '{folder_name}' 時發生非預期錯誤: {e}")
         return None

def extract_folder_id_from_url(url):
    """從 Google Drive 資料夾網址中提取 ID"""
    match = re.search(r'/folders/([a-zA-Z0-9_-]+)', url)
    if match:
        return match.group(1)
    # 嘗試匹配另一種可能的 URL 格式 (如果 ID 直接在末尾)
    match = re.search(r'id=([a-zA-Z0-9_-]+)', url)
    if match:
        return match.group(1)
    # 如果看起來不像 URL，假設它可能直接是 ID
    if re.match(r'^[a-zA-Z0-9_-]+$', url):
        return url
    return None

def trash_items_in_folder(folder_id, folder_name):
    """將指定資料夾內的所有項目 (檔案與子資料夾) 移至垃圾桶，
       **僅當執行者是該項目的擁有者時**"""
    global drive_service, DELAY_BETWEEN_REQUESTS, user_email # 確保 user_email 可用
    print(f"\n📄 正在列出資料夾 '{folder_name}' (ID: {folder_id}) 中的所有項目...")
    items_to_process = []
    page_token = None
    items_processed = 0
    items_trashed_count = 0
    items_skipped_count = 0 # 新增：計算因非擁有者而跳過的項目
    items_failed_count = 0

    # --- 第一階段：列出所有項目及其擁有者 ---
    print("  正在獲取項目列表及其擁有者資訊...")
    items_with_owners = []
    fetch_progress = tqdm(desc="  獲取項目資訊", unit=" 批")
    while True:
        try:
            time.sleep(DELAY_BETWEEN_REQUESTS)
            param = {
                'q': f"'{folder_id}' in parents and trashed = false",
                # **** 重要：增加 fields='files(id, name, mimeType, owners)' ****
                'fields': "nextPageToken, files(id, name, mimeType, owners)",
                'pageSize': 100, # 減少 pageSize 可能有助於避免超時，但會增加請求次數
                'pageToken': page_token
            }
            response = drive_service.files().list(**param).execute()
            files = response.get('files', [])
            items_with_owners.extend(files)
            fetch_progress.update(1) # 更新進度條
            page_token = response.get('nextPageToken')
            if not page_token:
                break
        except HttpError as error:
            fetch_progress.close()
            print(f"\n  ❌ 列出資料夾 '{folder_name}' 中的項目時發生 API 錯誤: {error}")
            # 返回部分成功和失敗計數
            return {'success': items_trashed_count, 'skipped': items_skipped_count, 'failed': items_failed_count + len(items_with_owners) - items_processed}
        except Exception as e:
            fetch_progress.close()
            print(f"\n  ❌ 列出資料夾 '{folder_name}' 中的項目時發生非預期錯誤: {e}")
            return {'success': items_trashed_count, 'skipped': items_skipped_count, 'failed': items_failed_count + len(items_with_owners) - items_processed}
    fetch_progress.close()

    if not items_with_owners:
        print(f"  ✅ 資料夾 '{folder_name}' 中沒有需要處理的項目。")
        return {'success': 0, 'skipped': 0, 'failed': 0}

    print(f"  🔍 共找到 {len(items_with_owners)} 個項目，開始檢查擁有者並移至垃圾桶...")

    # --- 第二階段：處理項目 ---
    for item in tqdm(items_with_owners, desc=f"  處理項目中", unit="個"):
        items_processed += 1
        item_id = item['id']
        item_name = item.get('name', f'未命名項目 (ID: {item_id})')
        item_type = "資料夾" if item.get('mimeType') == 'application/vnd.google-apps.folder' else "檔案"
        item_owners = item.get('owners', [])

        # **** 核心檢查：判斷執行者是否為擁有者 ****
        is_owner = False
        if item_owners:
            # 通常只有一個擁有者，但 API 返回列表
            owner_email = item_owners[0].get('emailAddress')
            if owner_email == user_email:
                is_owner = True
            # else: tqdm.write(f"    ⏭️ 跳過 {item_type} '{item_name}' (ID: {item_id})，擁有者是 {owner_email}，不是您 ({user_email})。")
        else:
             # 理論上不應發生，但以防萬一
             tqdm.write(f"    ⚠️ 警告：無法獲取 {item_type} '{item_name}' (ID: {item_id}) 的擁有者資訊，將跳過。")

        if is_owner:
            # 只有擁有者才嘗試移至垃圾桶
            try:
                time.sleep(DELAY_BETWEEN_REQUESTS)
                drive_service.files().update(
                    fileId=item_id,
                    body={'trashed': True}
                    # 可考慮增加 fields='id' 減少回應大小
                ).execute()
                items_trashed_count += 1
            except HttpError as error:
                tqdm.write(f"    ❌ 將您擁有的 {item_type} '{item_name}' (ID: {item_id}) 移至垃圾桶時發生 API 錯誤: {error}")
                items_failed_count += 1
            except Exception as e:
                tqdm.write(f"    ❌ 將您擁有的 {item_type} '{item_name}' (ID: {item_id}) 移至垃圾桶時發生非預期錯誤: {e}")
                items_failed_count += 1
        else:
            items_skipped_count += 1 # 增加跳過計數

    print(f"\n  📊 項目處理結果：成功移至垃圾桶 {items_trashed_count} 個 (您是擁有者)，跳過 {items_skipped_count} 個 (您非擁有者)，失敗 {items_failed_count} 個。")
    return {'success': items_trashed_count, 'skipped': items_skipped_count, 'failed': items_failed_count}

def empty_trash():
    """清空 Google Drive 垃圾桶"""
    global drive_service
    try:
        drive_service.files().emptyTrash().execute()
        print("✅ Google Drive 垃圾桶已清空。")
        return True
    except HttpError as error:
        print(f"❌ 清空垃圾桶時發生 API 錯誤: {error}")
        return False
    except Exception as e:
        print(f"❌ 清空垃圾桶時發生非預期錯誤: {e}")
        return False

# --- 主執行流程 ---

if __name__ == "__main__":
    print("🚀 開始執行移動個人資料夾內所有項目至垃圾桶腳本 🚀")
    print("⚠️ 警告：此腳本將您帳號對應的分類資料夾內的所有項目移至垃圾桶！")

    if not TARGET_BASE_FOLDER_ID or TARGET_BASE_FOLDER_ID == 'YOUR_TARGET_BASE_FOLDER_ID_HERE':
         sys.exit("❌ 請先在 '使用者設定' 中填入 'TARGET_BASE_FOLDER_ID' (通常是 '待刪除檔案' 資料夾的 ID)。")

    TARGET_BASE_FOLDER_ID = extract_folder_id_from_url(TARGET_BASE_FOLDER_ID)

    if not TARGET_BASE_FOLDER_ID:
        sys.exit(f"❌ 無法從輸入 '{TARGET_BASE_FOLDER_ID}' 中提取有效的 Google Drive 資料夾 ID。請檢查輸入是否為正確的資料夾網址或 ID。")

    if not authenticate_and_build_service():
        sys.exit("❌ 無法繼續執行，請檢查驗證問題。")

    if not user_email:
         sys.exit("❌ 無法取得目前使用者 Email，無法繼續。")

    # 1. 驗證總資料夾 ID 並取得名稱
    print(f"\n🔍 正在驗證總資料夾 ID: {TARGET_BASE_FOLDER_ID}...")
    target_base_folder_name = get_folder_name(TARGET_BASE_FOLDER_ID)

    if not target_base_folder_name:
        print(f"❌ 無法驗證或存取總資料夾 ID '{TARGET_BASE_FOLDER_ID}'。")
        print(f"   請確認 ID 是否正確、資料夾是否存在且未被移入垃圾桶，以及您是否有權限存取。")
        sys.exit("無法繼續執行。")
    else:
        print(f"✅ 總資料夾確認: '{target_base_folder_name}' (ID: {TARGET_BASE_FOLDER_ID})")


    # 2. 在總資料夾內找到使用者 Email 對應的資料夾 ID
    user_folder_name = user_email
    print(f"\n🔍 正在總資料夾 '{target_base_folder_name}' 內尋找您的個人資料夾 '{user_folder_name}'...")
    user_folder_id = find_folder_id_by_name(TARGET_BASE_FOLDER_ID, user_folder_name)

    items_moved_to_trash = False # 標記是否有項目被移動

    if not user_folder_id:
        print(f"✅ 在 '{target_base_folder_name}' 中找不到名為 '{user_folder_name}' 的資料夾。可能已被移至垃圾桶或不存在。")
    else:
        print(f"✅ 找到您的個人資料夾 '{user_folder_name}' (ID: {user_folder_id})")
        if EMAIL_VERIFY != user_email: # 檢查 Email
            print("\n🚫 操作已取消。輸入的 Email 不符或未輸入。")
            sys.exit("腳本已中止。")

        # 4. 執行移動項目至垃圾桶 (現在會檢查擁有者)
        print("\n⏳ 正在將個人資料夾內 **您擁有** 的項目移至垃圾桶...")
        trash_result = trash_items_in_folder(user_folder_id, user_folder_name)
        items_moved_to_trash = trash_result['success'] > 0 # 只有成功移動的才算

        if trash_result['skipped'] > 0:
             print(f"\nℹ️ {trash_result['skipped']} 個項目因您不是擁有者而被跳過。")
        if trash_result['failed'] > 0:
             print(f"\n⚠️ 移動項目過程中發生 {trash_result['failed']} 個錯誤。請檢查上方訊息。")

        if items_moved_to_trash:
             print("\n✅ 您擁有的項目移動至垃圾桶操作完成。")
        elif trash_result['skipped'] > 0 and trash_result['failed'] == 0:
             print("\n✅ 資料夾內沒有您擁有的項目需要移動。")
        elif trash_result['failed'] == 0 and trash_result['skipped'] == 0:
             # 這種情況對應 trash_items_in_folder 開頭的 "沒有需要處理的項目"
             pass # 不重複打印訊息
        # else: # 有失敗的情況，上面已經打印過錯誤信息

    if AUTO_CLEAN:
        print("\n--- 自動清空垃圾桶設定已啟用 ---")
        # 確保 drive_service 和 user_email 仍然可用
        if 'drive_service' not in globals() or drive_service is None or 'user_email' not in globals() or not user_email:
            print("⚠️ 雲端服務或使用者 Email 資訊遺失，無法自動清空垃圾桶。")
        else:
            empty_trash() # 呼叫清空函數
    else:
         if items_moved_to_trash: # 只有在移動過項目時才提示
             print("\nℹ️ 自動清空垃圾桶未啟用。您可以手動前往 Google Drive 清空垃圾桶。")


    print("\n🏁 ========== 移動項目腳本執行完畢 ========== 🏁")


# 腳本說明：清空分類後的個人資料夾

## 用途

*   此腳本用於將 Google Drive 中，位於指定「總資料夾」（例如 "待刪除檔案"）下，以**執行腳本者 Email 命名**的那個子資料夾內的**所有項目**（包含檔案和子資料夾）移至 Google Drive 的垃圾桶。
*   **核心功能**：腳本會檢查每個項目的擁有者，**僅會嘗試將屬於執行腳本者本人擁有的項目移至垃圾桶**。不屬於執行者的項目會被跳過。
*   使用者可以選擇是否在移動完成後**自動清空垃圾桶**。
*   包含一個**Email 確認機制**，要求使用者輸入自己的 Email，以防止誤操作。

## 與「依擁有者分類資料夾下檔案」的關係

*   此腳本通常作為 **「依擁有者分類資料夾下檔案」腳本的後續步驟**。
*   「依擁有者分類資料夾下檔案」腳本會將共享空間中的檔案，依照擁有者 Email 分類到「總資料夾」下的各個 Email 子資料夾中。
*   執行完分類後，每個使用者可以運行**此腳本**，指定相同的「總資料夾」，腳本會自動找到該使用者的 Email 資料夾，並將裡面**屬於該使用者自己**的檔案和資料夾移至垃圾桶，方便使用者清理自己擁有的檔案。

## 適用情境

*   在執行「依擁有者分類資料夾下檔案」後，使用者希望快速清理或刪除被歸類到自己 Email 資料夾下的檔案。
*   需要將特定資料夾內**屬於自己**的大量檔案和子資料夾批次移至垃圾桶時。

## 權限要求

*   **執行者**：執行此腳本的 Google 帳號需要：
    *   對指定的「總資料夾」擁有至少**檢視者 (Viewer)** 權限，以便能找到以自己 Email 命名的子資料夾。
    *   對其 Email 命名的子資料夾擁有至少**編輯者 (Editor)** 權限（通常在分類後會自動擁有）。
    *   **最重要**：對於 Email 子資料夾內要被移至垃圾桶的項目，執行者**必須是該項目的擁有者 (Owner)**。腳本會檢查此點，非擁有者的項目會被跳過。
    *   如果啟用了「自動清除垃圾桶」，則需要清空垃圾桶的權限（通常帳號擁有者都有）。

## 警告說明

*   **移至垃圾桶**：此腳本會將您 Email 資料夾內**您所擁有**的項目**移至垃圾桶**。這不是立即永久刪除，但若後續清空垃圾桶，則無法復原。
*   **僅限擁有者**：再次強調，腳本**只會處理您擁有的項目**。如果您 Email 資料夾內有其他人擁有的檔案（理論上分類腳本後不應發生，但可能手動加入），這些檔案會被保留在原位。
*   **Email 確認**：請務必在執行前，於「操作確認」欄位**正確輸入您當前登入 Colab/執行腳本的 Google 帳號 Email**。這是防止誤操作的重要保險步驟。輸入錯誤或留空將導致腳本中止。
*   **目標資料夾**：請確保「總資料夾」的網址或 ID 輸入正確，指向先前分類腳本使用的那個總資料夾。
*   **自動清空垃圾桶**：如果啟用此選項，移至垃圾桶的項目將**立即被永久刪除**，無法復原。請謹慎啟用。
*   **API 限制**：處理大量項目時，仍可能遇到 Google Drive API 的速率限制或超時。腳本包含延遲，但錯誤仍可能發生。